In [36]:
import os
import asyncio
import aiohttp
import json
import re
from random import shuffle
import random
import pandas as pd
from dataclasses import dataclass

In [37]:
from card_utils import read_all_game_data, Card, PlayerCards, PlayerStats

In [38]:
random.seed(42)

In [39]:
from deck_rate_fetcher import fetch_deck_rating

In [40]:
import dotenv
dotenv.load_dotenv()

import nest_asyncio
nest_asyncio.apply()

In [41]:
from langchain_openai import AzureChatOpenAI
from openai import AsyncAzureOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [42]:
game_cards_list, player_cards_list, player_stats, player_tags_list = read_all_game_data()

In [43]:
top_decks_per_card = pd.read_json("data/top_decks_per_card.json").to_dict()

In [44]:
print(top_decks_per_card)

{'Knight': {0: 'Graveyard,Knight,Ice Wizard,Witch,Goblin Hut,Vines,Poison,Barbarian Barrel', 1: 'Balloon,Miner,Little Prince,Knight,Bats,Goblin Cage,Rage,Arrows', 2: 'Mortar,Bandit,Knight,Archers,Ice Spirit,Inferno Tower,Rocket,Arrows', 3: 'Three Musketeers,Battle Ram,Royal Ghost,Bandit,Knight,Hunter,Elixir Collector,Barbarian Barrel', 4: 'Graveyard,Knight,Ice Golem,Bowler,Electro Wizard,Inferno Tower,Poison,The Log'}, 'Archers': {0: 'X Bow,Ice Golem,Archers,Skeletons,Fire Spirit,Tesla,Fireball,The Log', 1: 'Graveyard,Knight,Mega Minion,Archers,Skeletons,Tombstone,Poison,The Log', 2: 'Golem,Bandit,Night Witch,Baby Dragon,Archers,Skeletons,Lightning,The Log', 3: 'Mortar,Knight,Archers,Goblins,Ice Spirit,Rocket,Tornado,The Log', 4: 'Mortar,Knight,Princess,Archers,Skeletons,Ice Spirit,Poison,The Log'}, 'Goblins': {0: 'Giant,Electro Wizard,Night Witch,Mega Minion,Goblins,Tombstone,Poison,Zap', 1: 'Giant,Hog Rider,Musketeer,Minions,Goblins,Skeletons,Fireball,Zap', 2: 'Three Musketeers,Battl

In [45]:
for card_name in top_decks_per_card:
    for key in top_decks_per_card[card_name].keys():
        deck_list = top_decks_per_card[card_name][key].split(',')
        shuffle(deck_list)
        top_decks_per_card[card_name][key] = deck_list

In [46]:
print(top_decks_per_card)

{'Knight': {0: ['Witch', 'Goblin Hut', 'Poison', 'Barbarian Barrel', 'Ice Wizard', 'Vines', 'Graveyard', 'Knight'], 1: ['Knight', 'Arrows', 'Little Prince', 'Balloon', 'Bats', 'Rage', 'Goblin Cage', 'Miner'], 2: ['Archers', 'Inferno Tower', 'Knight', 'Ice Spirit', 'Bandit', 'Rocket', 'Arrows', 'Mortar'], 3: ['Royal Ghost', 'Battle Ram', 'Three Musketeers', 'Barbarian Barrel', 'Knight', 'Elixir Collector', 'Hunter', 'Bandit'], 4: ['Inferno Tower', 'Ice Golem', 'The Log', 'Bowler', 'Knight', 'Graveyard', 'Poison', 'Electro Wizard']}, 'Archers': {0: ['Tesla', 'Skeletons', 'Fireball', 'Fire Spirit', 'X Bow', 'The Log', 'Ice Golem', 'Archers'], 1: ['Archers', 'Knight', 'Graveyard', 'The Log', 'Skeletons', 'Mega Minion', 'Poison', 'Tombstone'], 2: ['Skeletons', 'The Log', 'Lightning', 'Night Witch', 'Archers', 'Golem', 'Baby Dragon', 'Bandit'], 3: ['Archers', 'Ice Spirit', 'The Log', 'Knight', 'Tornado', 'Mortar', 'Rocket', 'Goblins'], 4: ['Ice Spirit', 'Skeletons', 'Knight', 'Princess', 'Th

In [47]:
@dataclass
class LLMModel:
    name: str
    instance: AzureChatOpenAI | AsyncAzureOpenAI

In [48]:
llm_gpt3 = LLMModel(
    name="gpt-35-turbo",
    instance=AzureChatOpenAI(
        model_name="gpt-35-turbo",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
)

llm_gpt4o = LLMModel(
    name="gpt-4o",
    instance=AzureChatOpenAI(
        model_name="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
)

llm_gpt5 = LLMModel(
    name="gpt-5-chat",
    instance=AzureChatOpenAI(
        model_name="gpt-5-chat",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
)

In [ ]:
class SimpleAIMessage:
    """Clase simple para empaquetar la respuesta con un atributo .content"""
    def __init__(self, content):
        self.content = content

# Adaptador para AsyncAzureOpenAI de openai
class OpenAIClientAdapter:
    def __init__(self, client: AsyncAzureOpenAI, model_name: str, temperature: float = 0):
        self.client = client
        self.model_name = model_name
        self.temperature = temperature
        self.
        print(f"Adaptador creado para el modelo: {self.model_name}")

    def _convert_lc_messages_to_dict(self, messages):
        output_messages = []
        for msg in messages:
            role = msg.type
            if role == "human":
                role = "user"
            
            output_messages.append({"role": role, "content": msg.content})
        return output_messages

    async def ainvoke(self, messages):
        try:
            dict_messages = self._convert_lc_messages_to_dict(messages)
            
            response = await self.client.chat.completions.create(
                model=self.model_name,
                messages=dict_messages,
                temperature=self.temperature
            )
            
            content = response.choices[0].message.content
            
            return SimpleAIMessage(content=content)
        
        except Exception as e:
            print(f"Error en el adaptador de OpenAI ({self.model_name}): {e}")
            return SimpleAIMessage(content=None)

In [50]:
print("Creando clientes y adaptadores...")

async_openai_client = AsyncAzureOpenAI(
    azure_endpoint=dotenv.get_key(dotenv.find_dotenv(), "AZURE_OPENAI_ENDPOINT"),
    api_key=dotenv.get_key(dotenv.find_dotenv(), "AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview"
)

llm_grok_adapter = LLMModel(
    name="grok-4-fast-non-reasoning",
    instance=OpenAIClientAdapter(
        client=async_openai_client,
        model_name="grok-4-fast-non-reasoning",
        temperature=0
    )
)

llm_deepseek_adapter = LLMModel(
    name="DeepSeek-V3.1",
    instance=OpenAIClientAdapter(
        client=async_openai_client,
        model_name="DeepSeek-V3.1",
        temperature=0
    )
)

llm_llama_adapter = LLMModel(
    name="Llama-3.3-70B-Instruct",
    instance=OpenAIClientAdapter(
        client=async_openai_client,
        model_name="Llama-3.3-70B-Instruct",
        temperature=0
    )
)

Creando clientes y adaptadores...
Adaptador creado para el modelo: grok-4-fast-non-reasoning
Adaptador creado para el modelo: DeepSeek-V3.1
Adaptador creado para el modelo: Llama-3.3-70B-Instruct


### Test

In [51]:
import logging
import time
import sys

LOG_LEVEL = logging.INFO 

logging.basicConfig(
    level=LOG_LEVEL,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("logs/app_run{}.log".format(time.strftime('%Y%m%d_%H%M%S'))), 
        logging.StreamHandler(sys.stdout) 
    ],
    force=True 
)

logging.info("Logger configurado exitosamente.")

2025-11-02 23:25:19,065 - root - INFO - Logger configurado exitosamente.


In [52]:
with open("prompts/human_prompt_no_context.txt", "r", encoding="utf-8") as f:
    human_prompt_no_context = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_no_context = human_prompt_no_context.replace("\n", " ").replace('"', '\\"')
    human_prompt_no_context = human_prompt_no_context.replace("$", "{").replace("%", "}")

with open("prompts/human_prompt_context.txt", "r", encoding="utf-8") as f:
    human_prompt_context = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_context = human_prompt_context.replace("\n", " ").replace('"', '\\"')
    human_prompt_context = human_prompt_context.replace("$", "{").replace("%", "}")

with open("prompts/system_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

In [53]:
with open("prompts/human_prompt_no_context_lite.txt", "r", encoding="utf-8") as f:
    human_prompt_no_context_lite = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_no_context_lite = human_prompt_no_context_lite.replace("\n", " ").replace('"', '\\"')
    human_prompt_no_context_lite = human_prompt_no_context_lite.replace("$", "{").replace("%", "}")

with open("prompts/human_prompt_context_lite.txt", "r", encoding="utf-8") as f:
    human_prompt_context_lite = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_context_lite = human_prompt_context_lite.replace("\n", " ").replace('"', '\\"')
    human_prompt_context_lite = human_prompt_context_lite.replace("$", "{").replace("%", "}")

In [54]:
with open("prompts/human_prompt_no_context_semi_lite.txt", "r", encoding="utf-8") as f:
    human_prompt_no_context_semi_lite = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_no_context_semi_lite = human_prompt_no_context_semi_lite.replace("\n", " ").replace('"', '\\"')
    human_prompt_no_context_semi_lite = human_prompt_no_context_semi_lite.replace("$", "{").replace("%", "}")

with open("prompts/human_prompt_context_semi_lite.txt", "r", encoding="utf-8") as f:
    human_prompt_context_semi_lite = f.read().replace("{", "{{").replace("}", "}}")
    human_prompt_context_semi_lite = human_prompt_context_semi_lite.replace("\n", " ").replace('"', '\\"')
    human_prompt_context_semi_lite = human_prompt_context_semi_lite.replace("$", "{").replace("%", "}")

In [55]:
score_mapping = {
    "RIP": 0,
    "Bad": 1,
    "Mediocre": 2,
    "Good": 3,
    "Great!": 4,
    "Godly!": 5,
}

In [56]:
def process_deck_rating(rating: str) -> dict:
    rating = rating.strip()
    rating = rating.split(" ")
    ratings = {
        "Attack": score_mapping.get(rating[1], -1),
        "Defense": score_mapping.get(rating[3], -1),
        "Synergy": score_mapping.get(rating[5], -1),
        "Versatility": score_mapping.get(rating[7], -1),
        "F2P score": score_mapping.get(rating[10], -1),
    }
    return ratings

In [57]:
import re
import json

def parse_json_safe(player_cards, llm_response, llm_name, with_context: bool):
    player_tag = player_cards.tag[1:]
    deck_cards = player_cards.deck_cards
    deleted_cards = player_cards.deleted_cards or []

    try:
        s = (llm_response or "").strip()

        m = re.search(r"```(?:json)?\\s*([\\s\\S]*?)\\s*```", s, flags=re.IGNORECASE)
        if m:
            candidate = m.group(1).strip()
        else:
            candidate = s

        first_brace = candidate.find("{")
        first_bracket = candidate.find("[")
        starts = [i for i in (first_brace, first_bracket) if i != -1]
        
        if not starts:
            error_msg = f"! Error de parseo (Usuario: {player_tag}, LLM: {llm_name}, Ctx: {with_context}): No se encontró JSON."
            logging.warning(error_msg)
            return None, error_msg

        start = min(starts)
        end = max(candidate.rfind("}"), candidate.rfind("]"))
        
        if end == -1 or end < start:
            error_msg = f"! Error de parseo (Usuario: {player_tag}, LLM: {llm_name}, Ctx: {with_context}): JSON mal formado."
            logging.warning(error_msg)
            return None, error_msg
            
        candidate = candidate[start:end+1]
        parsed = json.loads(candidate)

        if isinstance(parsed, dict):
            parsed["user_id"] = player_tag
            parsed["deleted"] = list(map(lambda c: c.name, deleted_cards))
            parsed["original"] = list(map(lambda c: c.name, deck_cards))
            parsed["llm"] = llm_name
            parsed["with_context"] = with_context
        else:
            parsed = {
                "user_id": player_tag,
                "data": parsed,
                "deleted": list(map(lambda c: c.name, deleted_cards)),
                "llm": llm_name,
                "with_context": with_context
            }

        original_deck_rating_string = fetch_deck_rating(parsed["original"] + parsed["deleted"])
        selected_deck_rating_string = fetch_deck_rating(parsed["original"] + parsed["seleccion"])

        original_deck_ratings = process_deck_rating(original_deck_rating_string)
        selected_deck_ratings = process_deck_rating(selected_deck_rating_string)

        parsed["original_deck_rating"] = original_deck_ratings
        parsed["selected_deck_rating"] = selected_deck_ratings

        total_original_deck_rating = sum(original_deck_ratings.values())
        total_selected_deck_rating = sum(selected_deck_ratings.values())

        parsed["total_original_deck_rating"] = total_original_deck_rating
        parsed["total_selected_deck_rating"] = total_selected_deck_rating

        parsed["was_improved"] = total_selected_deck_rating >= total_original_deck_rating

        return parsed, None

    except json.JSONDecodeError as e:
        error_msg = f"X Error JSONDecode (Usuario: {player_tag}, LLM: {llm_name}, Ctx: {with_context}): {e}"
        logging.error(error_msg)
        return None, error_msg
    except Exception as e:
        error_msg = f"! Error inesperado (Usuario: {player_tag}, LLM: {llm_name}, Ctx: {with_context}): {e}"
        logging.error(error_msg)
        return None, error_msg

In [58]:
def write_prompt_file(string, user_id):
    with open(f"final_prompts_lite/user_{user_id}.txt", "w", encoding="utf-8") as f:
        f.write(string)

In [59]:
async def process_user_recommendation(
    llm, 
    player_cards, 
    system_prompt_str, 
    human_prompt_context_str,
    human_prompt_no_context_str,
    with_context: bool,
    valid_f,
    raw_f,
    file_lock
):
    player_tag = player_cards.tag[1:]
    available_cards = player_cards.available_cards
    deck_cards = player_cards.deck_cards

    available_cards_json = json.dumps(list(map(lambda c: c.name, available_cards)))
    deck_cards_json = json.dumps(list(map(lambda c: c.name, deck_cards)))

    try:
        if with_context:
            string_top_decks = "<Top 5 Decks para usar con cada una de las CARTAS_DISPONIBLES>\\n"
            
            for card in deck_cards:
                card_name = card.name
                string_top_decks += f'  <CardGroup name="{card_name}">\\n'
                decks_for_card = top_decks_per_card.get(card_name, {}) 
                for deck_key in decks_for_card.keys():
                    deck_list = decks_for_card[deck_key]
                    string_top_decks += "    <Deck>\\n"
                    for individual_card_name in deck_list:
                        string_top_decks += f'      <Card>{individual_card_name.strip()}</Card>\\n'
                    string_top_decks += "    </Deck>\\n"
                string_top_decks += f'  </CardGroup>\\n'
            string_top_decks += "</TopDecks>"

            rendered_human_prompt = human_prompt_context_str.format(
                TOP_DECKS=string_top_decks,
                CARTAS_DISPONIBLES=available_cards_json,
                CARTAS_SELECCIONADAS=deck_cards_json
            )
        else:
            rendered_human_prompt = human_prompt_no_context_str.format(
                CARTAS_DISPONIBLES=available_cards_json,
                CARTAS_SELECCIONADAS=deck_cards_json
            )
        
        prompt_filename_tag = f"{player_tag}{'_ctx' if with_context else '_noctx'}"
        write_prompt_file(rendered_human_prompt, prompt_filename_tag)

    except Exception as e:
        logging.error(f"! Error al renderizar prompt para usuario {player_tag} (Ctx: {with_context}): {e}")
        return False

    messages = [
        SystemMessage(content=system_prompt_str),
        HumanMessage(content=rendered_human_prompt),
    ]

    try:
        ai_msg = await llm.instance.ainvoke(messages)
        response_content = ai_msg.content

        parsed_result, parse_error = parse_json_safe(player_cards, response_content, llm.name, with_context)
        
        raw_log_data = {
            "user_id": player_tag,
            "llm": llm.name,
            "with_context": with_context,
            "raw_response": response_content,
            "parsed_successfully": (parsed_result is not None),
            "parse_error": parse_error
        }

        async with file_lock:
            raw_f.write(json.dumps(raw_log_data, ensure_ascii=False) + '\n')
            raw_f.flush()
            
            if parsed_result:
                valid_f.write(json.dumps(parsed_result, ensure_ascii=False) + '\n')
                valid_f.flush()

        if parsed_result:
            logging.info(f"Éxito: Usuario {player_tag} (LLM: {llm.name}, Ctx: {with_context})")
            return True
        else:
            return False

    except Exception as e:
        logging.error(f"! Error API (Usuario: {player_tag}, LLM: {llm.name}, Ctx: {with_context}): {e}")
        raw_log_data = {
            "user_id": player_tag,
            "llm": llm.name,
            "with_context": with_context,
            "raw_response": None,
            "parsed_successfully": False,
            "parse_error": f"API Error: {e}"
        }
        async with file_lock:
            raw_f.write(json.dumps(raw_log_data, ensure_ascii=False) + '\n')
            raw_f.flush()
            
        return False

In [60]:
def generate_human_prompt(human_prompt_template, top_decks, available_cards, selected_cards):
    rendered_human_prompt = human_prompt_template.format(
        TOP_DECKS=top_decks,
        CARTAS_DISPONIBLES=available_cards,
        CARTAS_SELECCIONADAS=selected_cards
    )
    return rendered_human_prompt

In [61]:
def shuffle_and_remove_cards(player_cards_list, num_to_remove=4):
    updated_player_cards_list = []
    for player_cards in player_cards_list:
        
        available_copy = list(player_cards.available_cards)
        selected_copy = list(player_cards.deck_cards)

        shuffle(available_copy)
        shuffle(selected_copy)

        to_delete = selected_copy[-num_to_remove:]
        selected_for_prompt = selected_copy[:-num_to_remove]

        updated_player_cards = PlayerCards(
            tag=player_cards.tag,
            available_cards=available_copy,
            deck_cards=selected_for_prompt,
            deleted_cards=to_delete
        )
        updated_player_cards_list.append(updated_player_cards)
    
    return updated_player_cards_list


In [62]:
import time

async def run_all_models_async(
    player_cards_list, 
    models_list,
    system_prompt_str, 
    human_prompt_context_str,
    human_prompt_no_context_str,
    batch_size=50, 
    num_batches=None,
    starting_card_number=4
):
    """
    Orquesta la ejecución asíncrona para todos los LLMs y usuarios,
    controlando el tamaño de los lotes y guardando los resultados en JSONL.
    """
    selected_player_cards_list = shuffle_and_remove_cards(
        player_cards_list, 
        num_to_remove= 8 - starting_card_number
    )

    if num_batches is not None:
        total_to_process = batch_size * num_batches
        players_to_process = selected_player_cards_list[:total_to_process]
    else:
        players_to_process = selected_player_cards_list

    logging.info(f"Iniciando procesamiento para {len(players_to_process)} jugadores a través de {len(models_list)} LLMs (x2 con/sin contexto).")

    output_timestamp = time.strftime('%Y%m%d_%H%M%S')
    valid_results_filename = f"results/valid_results_{output_timestamp}.jsonl"
    raw_results_filename = f"results/raw_results_{output_timestamp}.jsonl"
    
    file_lock = asyncio.Lock()
    
    with open(valid_results_filename, 'a', encoding='utf-8') as valid_f, \
         open(raw_results_filename, 'a', encoding='utf-8') as raw_f:

        all_tasks = []
        for llm in models_list:
            logging.info(f"Creando tareas para: {llm.name}...")
            for player_cards in players_to_process:
                for with_context_flag in [True, False]:
                    task = process_user_recommendation(
                        llm,
                        player_cards, 
                        system_prompt_str, 
                        human_prompt_context_str,
                        human_prompt_no_context_str,
                        with_context_flag,
                        valid_f,
                        raw_f,
                        file_lock
                    )
                    all_tasks.append(task)
                
        logging.info(f"Total de tareas creadas: {len(all_tasks)}")
        
        all_results = []
        total_batches = (len(all_tasks) + batch_size - 1) // batch_size
        
        for i in range(0, len(all_tasks), batch_size):
            current_batch_num = (i // batch_size) + 1
            batch_tasks = all_tasks[i:i + batch_size]
            logging.info(f"\n--- Ejecutando Lote {current_batch_num} / {total_batches} (Tamaño: {len(batch_tasks)}) ---")
            
            batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)
            all_results.extend(batch_results)
            
            for idx, result in enumerate(batch_results):
                if isinstance(result, Exception):
                    logging.error(f"! Excepción en tarea del lote {current_batch_num}: {result}")
            
            await asyncio.sleep(1) 

    logging.info("\n--- Procesamiento de lotes completado. ---")
    
    valid_count = sum(1 for r in all_results if r is True)
    logging.info(f"Resultados válidos (parseados y guardados): {valid_count} de {len(all_tasks)}")

    return valid_results_filename, raw_results_filename

In [63]:
import time

models_to_run = [
    #llm_gpt3,
    #llm_gpt4o,
    #llm_gpt5,
    #llm_grok_adapter,
    llm_deepseek_adapter,
    llm_llama_adapter,
]

BATCH_SIZE = 10
NUM_BATCHES = 1

logging.info("Iniciando la ejecución asíncrona...")

valid_filename, raw_filename = await run_all_models_async(
    player_cards_list=player_cards_list,
    models_list=models_to_run,
    system_prompt_str=system_prompt,
    human_prompt_context_str=human_prompt_context_semi_lite,
    human_prompt_no_context_str=human_prompt_no_context_semi_lite,
    batch_size=BATCH_SIZE,
    num_batches=NUM_BATCHES
)

logging.info(f"Ejecución finalizada.")
logging.info(f"Resultados válidos guardados en: '{valid_filename}'")
logging.info(f"Resultados crudos guardados en: '{raw_filename}'")

output_filename = valid_filename

2025-11-02 23:25:19,112 - root - INFO - Iniciando la ejecución asíncrona...
2025-11-02 23:25:20,122 - root - INFO - Iniciando procesamiento para 10 jugadores a través de 2 LLMs (x2 con/sin contexto).
2025-11-02 23:25:20,123 - root - INFO - Creando tareas para: DeepSeek-V3.1...
2025-11-02 23:25:20,123 - root - INFO - Creando tareas para: Llama-3.3-70B-Instruct...
2025-11-02 23:25:20,124 - root - INFO - Total de tareas creadas: 40
2025-11-02 23:25:20,124 - root - INFO - 
--- Ejecutando Lote 1 / 4 (Tamaño: 10) ---
2025-11-02 23:25:21,419 - httpx - INFO - HTTP Request: POST https://victo-mhcmsfx4-eastus2.cognitiveservices.azure.com/openai/deployments/DeepSeek-V3.1/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2025-11-02 23:25:22,589 - root - INFO - Éxito: Usuario 2G22QVP89 (LLM: DeepSeek-V3.1, Ctx: False)
2025-11-02 23:25:22,591 - httpx - INFO - HTTP Request: POST https://victo-mhcmsfx4-eastus2.cognitiveservices.azure.com/openai/deployments/DeepSeek-V3.1/chat/completion

In [64]:
df = pd.read_json(output_filename, orient="records", lines=True)

In [65]:
df['correct_selection_count'] = df.apply(
    lambda row: sum(1 for item in row['deleted'] if item in row['seleccion']),
    axis=1
)
df.head()

,seleccion,user_id,deleted,original,llm,with_context,original_deck_rating,selected_deck_rating,total_original_deck_rating,total_selected_deck_rating,was_improved,correct_selection_count
0,"[Goblin Barrel, The Log, Inferno Tower, Minions]",2G22QVP89,"[Royal Recruits, Goblin Cage, Zappies, Arrows]","[Electro Spirit, Flying Machine, Golden Knight...",DeepSeek-V3.1,False,"{'Attack': 3, 'Defense': 5, 'Synergy': 1, 'Ver...","{'Attack': 4, 'Defense': 5, 'Synergy': 2, 'Ver...",15,18,True,0
1,"[Tornado, Ice Golem, Skeletons, Cannon]",LU2QQJU0Y,"[Musketeer, Minions, Mega Knight, Firecracker]","[Electro Wizard, Hog Rider, Executioner, The Log]",DeepSeek-V3.1,False,"{'Attack': 4, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 2, 'Defense': 5, 'Synergy': 2, 'Ver...",20,14,False,0
2,"[Poison, Skeletons, The Log, Goblin Gang]",999QV8RJU,"[The Log, Goblin Gang, Zap, Mega Knight]","[Wall Breakers, Miner, Cannon, Bats]",DeepSeek-V3.1,False,"{'Attack': 4, 'Defense': 5, 'Synergy': 5, 'Ver...","{'Attack': 3, 'Defense': 5, 'Synergy': 4, 'Ver...",20,17,False,2
3,"[Miner, Poison, Inferno Dragon, Goblin Gang]",VL0QCY9R8,"[Zap, Mega Knight, Bandit, Spear Goblins]","[Firecracker, Bats, Skeletons, Skeleton Barrel]",DeepSeek-V3.1,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 4, 'Defense': 5, 'Synergy': 5, 'Ver...",20,22,True,0
4,"[Miner, Poison, Goblin Gang, Zap]",L0092LLGP,"[Bandit, Skeleton Barrel, Zap, Firecracker]","[Mega Knight, Spear Goblins, Bats, Skeletons]",DeepSeek-V3.1,True,"{'Attack': 3, 'Defense': 5, 'Synergy': 4, 'Ver...","{'Attack': 3, 'Defense': 5, 'Synergy': 5, 'Ver...",20,19,False,1


In [66]:
df['correct_selection_count'].groupby(df['llm']).value_counts()

llm                     correct_selection_count
DeepSeek-V3.1           0                          7
                        1                          1
                        2                          1
                        3                          1
Llama-3.3-70B-Instruct  0                          6
                        1                          1
Name: count, dtype: int64

In [67]:
res = (
    df.assign(wi=lambda d: d["was_improved"].astype(bool))
      .groupby("llm", as_index=False)
      .agg(total_mazos=("llm", "size"),
           mazos_mejorados=("wi", "sum"))
)
# Agregamos porcentaje de mazos mejorados
res['porcentaje_mejorados'] = (res['mazos_mejorados'] / res['total_mazos']) * 100
print(res)

                      llm  total_mazos  mazos_mejorados  porcentaje_mejorados
0           DeepSeek-V3.1           10                5             50.000000
1  Llama-3.3-70B-Instruct            7                2             28.571429


In [68]:
df_raw = pd.read_json(raw_filename, orient="records", lines=True)

In [69]:
# contamos cuantos validos e invalidos hay en raw_data para cada llm
df_raw['is_valid'] = df_raw['parsed_successfully'].astype(bool)
df_raw['llm'] = df_raw['llm'].astype('category')
df_raw['llm'].cat.categories
df_raw['is_valid'].groupby(df_raw['llm']).value_counts()
# Ponemos el porcentaje de validos por llm
df_raw.groupby('llm')['is_valid'].value_counts(normalize=True).mul(100)

/var/folders/jv/rmw91d8n0g15ldpn_dkk73rh0000gn/T/ipykernel_96029/17924982.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_raw['is_valid'].groupby(df_raw['llm']).value_counts()
/var/folders/jv/rmw91d8n0g15ldpn_dkk73rh0000gn/T/ipykernel_96029/17924982.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_raw.groupby('llm')['is_valid'].value_counts(normalize=True).mul(100)


llm                     is_valid
DeepSeek-V3.1           False       50.0
                        True        50.0
Llama-3.3-70B-Instruct  False       65.0
                        True        35.0
Name: proportion, dtype: float64

In [70]:
hola_serie = df_raw['raw_response'][(df_raw['user_id'] == '999QV8RJU') & (df_raw['llm'] == 'Llama-3.3-70B-Instruct')]

if not hola_serie.empty:
    texto_completo = hola_serie.iloc[0]
    
    print(texto_completo)
else:
    print("No se encontró esa fila.")

```json
{
  "seleccion": [
    "Zap",
    "Goblin Gang",
    "Mega Knight"
  ]
}
```
